In [ ]:
# Первая версия кода
import pandas as pd

def ads_performance(ads: pd.DataFrame) -> pd.DataFrame:
    ads = ads.groupby('ad_id').apply(
        lambda x:pd.Series({
            'clicked': (x['action'] == 'Clicked').sum(),
            'clicked_viewed': ((x['action'] == 'Clicked') | (x['action'] == 'Viewed')).sum()
        })
    ).reset_index()
    ads['ctr'] = ads.apply(
        lambda x: round(100 * x['clicked'] / x['clicked_viewed'], 2) if x['clicked_viewed'] >0 else 0, axis=1)

    ads = ads[['ad_id', 'ctr']].sort_values(by=['ctr', 'ad_id'], ascending=[False, True])

    return ads

In [ ]:
# Вторая версия, но не только пандас, а также нампай

import pandas as pd
import numpy as np

def ads_performance(ads: pd.DataFrame) -> pd.DataFrame:
    # Группируем по 'ad_id' и агрегируем с помощью функции count для подсчета значений
    ads_agg = ads.groupby('ad_id').agg(
        clicked=('action', lambda x: (x == 'Clicked').sum()),          # Подсчитываем количество 'Clicked'
        clicked_viewed=('action', lambda x: (x.isin(['Clicked', 'Viewed'])).sum())  # Подсчитываем 'Clicked' и 'Viewed'
    ).reset_index()

    # Рассчитываем CTR с использованием np.where для обработки деления на ноль
    ads_agg['ctr'] = np.where(ads_agg['clicked_viewed'] > 0,
                              round(100 * ads_agg['clicked'] / ads_agg['clicked_viewed'], 2),
                              0)

    # Сортируем результат по 'ctr' в порядке убывания и по 'ad_id' в порядке возрастания
    ads_agg = ads_agg[['ad_id', 'ctr']].sort_values(by=['ctr', 'ad_id'], ascending=[False, True])

    return ads_agg


In [ ]:
# Третия версия, только пандас

import pandas as pd

def ads_performance(ads: pd.DataFrame) -> pd.DataFrame:
    # Группируем по 'ad_id' и считаем количество "Clicked" и "Clicked or Viewed" событий
    ads_agg = ads.groupby('ad_id').agg(
        clicked=('action', lambda x: (x == 'Clicked').sum()),  # Подсчет "Clicked"
        clicked_viewed=('action', lambda x: x.isin(['Clicked', 'Viewed']).sum())  # Подсчет "Clicked" и "Viewed"
    ).reset_index()

    # Рассчитываем CTR с помощью apply
    ads_agg['ctr'] = ads_agg.apply(
        lambda x: round(100 * x['clicked'] / x['clicked_viewed'], 2) if x['clicked_viewed'] > 0 else 0,
        axis=1
    )

    # Сортируем по 'ctr' и 'ad_id'
    ads_agg = ads_agg[['ad_id', 'ctr']].sort_values(by=['ctr', 'ad_id'], ascending=[False, True])

    return ads_agg